# [데이터 수집 및 전처리]

In [16]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# shp(서울시 동별) 데이터

In [2]:
dong_shp = pd.read_csv('DATA_LSMD_ADM_SECT_UMD_서울_동별_prc.csv', encoding = 'cp949')
dong_shp_df = dong_shp.iloc[:, :2]
dong_shp_df.columns = ['EMD_CD', 'EMD_NM']
dong_shp_df

,EMD_CD,EMD_NM
0,11710107,가락동
1,11530103,가리봉동
2,11545101,가산동
3,11500104,가양동
4,11110146,가회동
...,...,...
462,11170101,후암동
463,11110150,훈정동
464,11230109,휘경동
465,11590105,흑석동


# 장애인콜택시 이용고객 목적지(동단위) Best100

### 1) 장애인콜택시 이용고객 목적지(동단위) Best100 - 크롤링

In [3]:
#2019.01.01 ~ 2019.12.31 장애인콜택시 이용고객 목적지(동단위) Best100데이터 수집 및 전처리

#수집 기간 정의
import datetime

days_range = [] #날짜 저장할 리스트

start = datetime.datetime.strptime("20190101", "%Y%m%d") #시작점 : 2019.01.01
end = datetime.datetime.strptime("20200101", "%Y%m%d") #끝점 : 2019.12.31 -> 범위 + 1
date_generated = [start + datetime.timedelta(days = x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y%m%d"))


#크롤러 작성
from tqdm.notebook import tqdm

temp = [] #날짜별 Best100 데이터 저장할 리스트

#데이터 불러오기
def get_data(url):
    response = requests.get(url).text
    df = pd.read_html(response)
    return df

#(날짜별) 포맷에 맞춰 url 크롤링
for date in tqdm(days_range):
    base_url = f'http://m.calltaxi.sisul.or.kr/api/open/newEXCEL0002.asp?key=29c3729265fa3b49c7d3e75147dc19a3&sDate={date}'
    df1 = get_data(base_url)
    
    for row in df1:
        row
    
    temp.append(row)

# 날짜별 데이터 하나의 데이터셋으로 저장
taxi_raw = pd.concat(temp, ignore_index = True)
taxi_raw

taxi_raw.to_csv('DATA_장애인콜택시 이용고객 목적지(동단위) Best100_raw.csv', encoding = 'cp949')

# => 20190101 ~ 20191231 raw데이터 저장 완료


### 2) 장애인콜택시 이용고객 목적지(동단위) Best100 전처리

In [4]:
# taxi 데이터 칼럼명 변경
taxi = taxi_raw.iloc[:]
taxi.columns = ['DATE', 'SI_NM', 'GU_NM', 'EMD_NM', 'TAXI_CNT']


# raw데이터에서 '서울특별시' 데이터만 출력해서 새로운 csv파일 저장
taxi_seoul = taxi[taxi['SI_NM'] == '서울특별시']
taxi_seoul.set_index("DATE", inplace = True)
taxi_seoul.to_csv('DATA_장애인콜택시 이용고객 목적지(동단위) Best100_Seoul_prc.csv', encoding = 'cp949')

In [5]:
# 택시 데이터 동별 그룹핑 - 택시 건수 더하기

taxi_seoul_df = pd.read_csv('DATA_장애인콜택시 이용고객 목적지(동단위) Best100_Seoul_prc.csv', encoding = 'cp949')
taxi_seoul_df2 = taxi_seoul_df.iloc[:, 3:]
taxi_dong = taxi_seoul_df2.groupby('EMD_NM').sum()
taxi_dong

taxi_dong.to_csv('DATA_장애인콜택시 이용고객 목적지(동단위) Best100_Seoul_dong_prc.csv', encoding = 'cp949')

#### 데이터파일명 : DATA_장애인콜택시 이용고객 목적지(동단위) Best100_Seoul_dong_(법정동수정)_prc.csv

-> 위의 데이터는 <DATA_장애인콜택시 이용고객 목적지(동단위) Best100_Seoul_dong_prc.csv> 데이터를 법정동 기준으로 전처리한 데이터

[법정동 변경 기준]
   
   step1) 행정동 -> 법정동으로 변경 
   
   step2) 하나의 행정동이 여러 개의 법정동으로 나눠질 경우 주민센터 기준 법정동 전처리

In [6]:
# 행정동 -> 법정동 변경 완료된 데이터 다시 전처리 진행

taxi_prc = pd.read_csv('DATA_장애인콜택시 이용고객 목적지(동단위) Best100_Seoul_dong_(법정동수정)_prc.csv', encoding = 'cp949')
taxi_prc_df = taxi_prc.iloc[:, :2]

taxi_prc_df2 = taxi_prc_df.groupby('EMD_NM').sum()
taxi_prc_df2

,TAXI_CNT
EMD_NM,
가락동,2774
가리봉동,92
가산동,156
가양동,13316
갈현동,130
...,...
회현동1가,1127
효제동,43
효창동,816


### 3) shp(서울시 동별) 데이터 + 장애인콜택시 목적지 Best100 데이터 JOIN

In [7]:
#shp(동별) 데이터 기준 콜택시 목적지 데이터 조인

taxi_df = pd.merge(left = dong_shp_df, right = taxi_prc_df2, how = 'left', on = 'EMD_NM')
taxi_df = taxi_df.fillna(0)
taxi_df.set_index('EMD_CD', inplace = True)
taxi_df
taxi_df.to_csv('DATA_장애인콜택시 이용고객 목적지(동단위) Best100_prc.csv', encoding = 'cp949')
# => '장애인콜택시 목적지 Best100' 최종 전처리 완료 데이터

# 장애인 구인 현황

### 1) 장애인 구인 현황 전처리

#### DATA_장애인 구인 현황_EMD_NM_prc.csv
-> 위의 데이터는 <DATA_장애인 구인 현황_raw.csv> 데이터에 동 이름과 위/경도 추가한 데이터

In [8]:
# 장애인 동별 구인현황 데이터 불러오기
job = pd.read_csv("DATA_장애인 구인 현황_EMD_NM_prc.csv", encoding = 'cp949')


# raw 데이터에서 동별 구인현황 추출
job_dong = job.groupby('EMD_NM')
job_dong_sum = job_dong['구인인원'].sum()
job_dong_df = pd.DataFrame(job_dong_sum)

# shp(서울시 동별) 데이터와 조인 후 저장
job_join = pd.merge(left = dong_shp_df, right = job_dong_df, how = 'right', on = 'EMD_NM')
job_join.to_csv("DATA_장애인 구인 현황_EMD_NM_(shp)1stJOIN_prc.csv", encoding = 'cp949')

#### DATA_장애인 구인 현황_EMD_NM_(shp)1stJOIN_(법정동수정)_prc.csv
-> 위의 데이터는 <DATA_장애인 구인 현황_EMD_NM_(shp)1stJOIN_prc.csv> 데이터를 법정동 기준으로 전처리한 데이터

[법정동 변경 기준]

step1) 행정동 -> 법정동으로 변경 

step2) 하나의 행정동이 여러 개의 법정동으로 나눠질 경우 주민센터 기준 법정동 전처리

In [9]:
# 행정동 -> 법정동 변경 완료된 데이터 다시 전처리 진행
# 동 수정 과정에서 동일한 동 데이터가 존재할 수 있기에 다시 동별로 구인 현황 sum

job_prc = pd.read_csv('DATA_장애인 구인 현황_EMD_NM_(shp)1stJOIN_(법정동수정)_prc.csv', encoding = 'cp949')
job_prc_df = job_prc.iloc[:, 1:3]

job_prc_df2 = job_prc_df.groupby('EMD_NM').sum()
job_prc_df2

,EMPLOYEENUM
EMD_NM,
가락동,3
가산동,27
개포동,3
거여동,4
계동,1
...,...
필동2가,1
한강로3가,2
한남동,2


### 2) shp(서울시 동별) 데이터 + 장애인 구인 현황 데이터 JOIN

In [10]:
# #shp(동별) 데이터 기준 장애인 구인 현황 데이터 조인 및 저장

job_df = pd.merge(left = dong_shp_df, right = job_prc_df2, how = 'left', on = 'EMD_NM')
job_df = job_df.fillna(0)
job_df.set_index('EMD_CD', inplace = True)
job_df.to_csv("DATA_장애인 구인 현황_prc.csv", encoding = 'cp949')

# 서울시 장애인 현황(장애유형별, 동별) 통계

#### DATA_서울시 장애인 현황(장애유형별, 동별) 통계_prc.csv
-> 위의 데이터는 <DATA_서울시 장애인 현황(장애유형별,동별) 통계_raw.txt> 데이터를 '지체/뇌병변/심장장애/호흡기장애'만 추출하여 동 이름 추가하고 장애인 수 카운트한 데이터

In [11]:
disabled_df = pd.read_csv('DATA_서울시 장애인 현황(장애유형별, 동별) 통계_prc.csv', encoding = 'cp949')
disabled_df

,EMD_NM,EMD_CD,DISABLED_NUM
0,가락동,11710107,1235
1,가리봉동,11530103,305
2,가산동,11545101,437
3,가양동,11500104,2654
4,가회동,11110146,97
...,...,...,...
207,효제동,11110162,158
208,효창동,11170119,172
209,후암동,11170101,627
210,휘경동,11230109,941


# 서울시 장애인시설 현황

#### DATA_서울시 장애인시설 현황_prc.csv
-> 위의 데이터는 <DATA_서울시 장애인시설 현황_raw.csv> 데이터를 QGIS에서 시각화 후 법정동별 개수 카운트한 데이터

In [12]:
welfare_df = pd.read_csv('DATA_서울시 장애인시설 현황_prc.csv', encoding = 'cp949')
welfare_df

,EMD_NM,EMD_CD,CENTER_CNT
0,가락동,11710107,0
1,가리봉동,11530103,2
2,가산동,11545101,0
3,가양동,11500104,4
4,가회동,11110146,0
...,...,...,...
462,후암동,11170101,1
463,훈정동,11110150,0
464,휘경동,11230109,1
465,흑석동,11590105,0


# 서울시 신한카드 장애인 복지카드 이용현황

#### DATA_서울시 신한카드 장애인 복지카드 이용현황_prc.csv
-> 위의 데이터는 '서울시 빅데이터 캠퍼스'가 보유한 Raw Data의 법정동별 이용횟수를 QGIS에서 시각화하여 반출. 이후 시각화한 자료를 카운트한 데이터

In [13]:
card_df = pd.read_csv('DATA_서울시 신한카드 장애인 복지카드 이용현황_prc.csv', encoding = 'cp949')
card_df

,EMD_NM,EMD_CD,CARD
0,가락동,11710107,3205
1,가리봉동,11530103,471
2,가산동,11545101,5397
3,가양동,11500104,1911
4,가회동,11110146,573
...,...,...,...
462,후암동,11170101,716
463,훈정동,11110150,0
464,휘경동,11230109,1386
465,흑석동,11590105,2740


# 대규모점포 인허가 정보

#### DATA_대규모점포 인허가 정보_prc.csv
-> 위의 데이터는 <DATA_대규모점포 인허가 정보_raw.csv> 데이터를 QGIS에서 시각화 후 법정동별로 카운트한 데이터

In [14]:
shop_df =  pd.read_csv('DATA_대규모점포 인허가 정보_prc.csv', encoding = 'cp949')
shop_df

,EMD_NM,EMD_CD,SHOP_CNT
0,가락동,11710107,6
1,가리봉동,11530103,0
2,가산동,11545101,6
3,가양동,11500104,4
4,가회동,11110146,0
...,...,...,...
462,후암동,11170101,0
463,훈정동,11110150,0
464,휘경동,11230109,0
465,흑석동,11590105,1


# 활용데이터 전체 JOIN

In [15]:
join1 = pd.merge(left = dong_shp_df, right = disabled_df.iloc[:, 1:], how = 'left', on = 'EMD_CD')
join2 = pd.merge(left = join1, right = shop_df.iloc[:, 1:], how = 'left', on = 'EMD_CD')
join3 = pd.merge(left = join2, right = taxi_df.iloc[:, 1:], how = 'left', on = "EMD_CD")
join4 = pd.merge(left = join3, right = job_df.iloc[:, 1:], how = 'left', on = 'EMD_CD')
join5 = pd.merge(left = join4, right = welfare_df.iloc[:, 1:], how = 'left', on = 'EMD_CD')
join6 = pd.merge(left = join5, right = card_df.iloc[:, 1:], how = 'left', on = 'EMD_CD')

# 결측치 처리 및 저장
total = join6.fillna(0)
total.set_index('EMD_CD', inplace = True)
total
total.to_csv('DATA_TOTAL DATASET.csv', encoding = 'cp949')